In [1]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
from groq import Groq

In [2]:
load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

OpenAI API Key exists and begins sk-proj-
Google API Key exists and begins AIzaSyCt


In [3]:
openai = OpenAI()

In [4]:
google.generativeai.configure()

In [5]:
client = Groq(api_key=os.environ.get("GROQ_API_KEY"))

In [6]:
gemini_via_openai_client = OpenAI(
    api_key=google_api_key, 
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

In [7]:
# messages_sent=[]
# gpt_messages = [f"Hi all. We are here to discuss on the topic {topic}. I disagree with this."]
# groq_messages = [f"Hi all. We are here to discuss on the topic {topic}. I disagree with this."]
# google_messages = [f"Hi all. We are here to discuss on the topic {topic}. I highly agree with this."]

In [8]:
def call_google():
    global google_system
    mess = ""
    for message in messages_sent:
        mess += f"\n {message}"

    messages = [{"role": "system", "content": google_system}, {"role": "user", "content": mess}]
    message = gemini_via_openai_client.chat.completions.create(
        messages=messages,
        model="gemini-2.0-flash-exp",
    )
    return message.choices[0].message.content



In [9]:
def call_groq():
    global groq_system
    mess = ""
    for message in messages_sent:
        mess += f"\n {message}"

    messages = [{"role": "system", "content": groq_system}, {"role": "user", "content": mess}]
    message = client.chat.completions.create(
        messages=messages,
        model="llama3-8b-8192",
    )
    return message.choices[0].message.content

In [10]:
def call_gpt():
    global gpt_system
    mess = ""
    for message in messages_sent:
        mess += f"\n {message}"

    messages = [{"role": "system", "content": gpt_system}, {"role": "user", "content": mess}]
    completion = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages
    )
    return completion.choices[0].message.content

In [11]:
import random
def next_user():    
    next_user = random.randint(0,2)
    return next_user

In [45]:


print(f"GPT:\n{gpt_messages[0]}\n")
print(f"Groq:\n{groq_messages[0]}\n")
print(f"Google:\n{google_messages[0]}\n")

next_user_next = random.randint(0, 2)
next_user_previous = -1  # Initialize with a value that won't match the first user

for i in range(20):
    next_user_next = random.randint(0, 2)

    # Ensure next_user_next is different from next_user_previous
    while next_user_next == next_user_previous:
        next_user_next = random.randint(0, 2)

    if next_user_next == 0:
        gpt_next = call_gpt()
        print(f"GPT:\n{gpt_next}\n")
        gpt_messages.append(gpt_next)
        messages_sent.append(gpt_next)
        next_user_previous = 0  # Update correctly
    elif next_user_next == 1:
        groq_next = call_groq()
        print(f"Groq:\n{groq_next}\n")
        groq_messages.append(groq_next)
        messages_sent.append(groq_next)
        next_user_previous = 1  # Update correctly
    else:
        google_next = call_google()
        print(f"Google:\n{google_next}\n")
        google_messages.append(google_next)
        messages_sent.append(google_next)
        next_user_previous = 2  # Update correctly

GPT:
Hi all. We are here to discuss on the topic Cell Phones Should Be Allowed In Schools. I disagree with this.

Groq:
Hi all. We are here to discuss on the topic Cell Phones Should Be Allowed In Schools. I disagree with this.

Google:
Hi all. We are here to discuss on the topic Cell Phones Should Be Allowed In Schools. I highly agree with this.

Groq:
You're all so predictable! You're missing the point, folks! Allowing cell phones in schools would mean chaos, distractions, and decreased academic productivity. It's not about embracing the "inevitable" or being "cool" or "with it." It's about putting the needs of students first and providing a focused learning environment. Can't you see that?

Google:
Oh, relax, Debbie Downer! A "focused learning environment"? Please. Have you seen a teenager try to focus on anything for more than five minutes without checking their phone? We might as well try teaching calculus to squirrels. At least with phones, they can Google the answers and pretend

In [11]:
import gradio as gr
import time
import re

# Global variables to store conversation history and system prompts
messages_sent = []
groq_messages = []
google_messages = []
google_system = ""
groq_system = ""

# Function to call Google AI (debater) and stream output
def call_google():
    global google_system, google_messages
    mess = "\n".join(google_messages)  # Convert list to string
    
    messages = [{"role": "system", "content": google_system}, {"role": "user", "content": mess}]
    message = gemini_via_openai_client.chat.completions.create(
        messages=messages,
        model="gemini-2.0-flash-exp",
    )
    
    google_response = message.choices[0].message.content
    google_messages.append(google_response)  # Store Google's response
    messages_sent.append(f"Google AI: {google_response}")
    
    yield f"Google AI: {google_response}"  # Streaming response
    time.sleep(5)

# Function to call Groq AI (debater) and stream output
def call_groq():
    global groq_system, groq_messages
    mess = "\n".join(groq_messages)  # Convert list to string
    
    messages = [{"role": "system", "content": groq_system}, {"role": "user", "content": mess}]
    message = client.chat.completions.create(
        messages=messages,
        model="llama3-8b-8192",
    )
    
    groq_response = message.choices[0].message.content
    groq_messages.append(groq_response)  # Store Groq's response
    messages_sent.append(f"Groq AI: {groq_response}")
    
    yield f"Groq AI: {groq_response}"  # Streaming response
    time.sleep(5)

# Function to call GPT-4o (judge) and stream output
def call_judge():
    global gpt_judge_system, messages_sent
    mess = "\n".join(messages_sent)  # Convert list to string
    
    messages = [{"role": "system", "content": gpt_judge_system}, {"role": "user", "content": mess}]
    
    completion = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages
    )
    
    judge_response = completion.choices[0].message.content
    messages_sent.append(f"Judge: {judge_response}")
    
    yield f"Judge (GPT-4o): {judge_response}"  # Streaming response
    time.sleep(5)
    
    # Extract probability and return progress values
    google_score, groq_score = extract_probabilities(judge_response)
    return google_score, groq_score

# GPT-4o as Judge - System Prompt (More Tolerant)
gpt_judge_system = """You are an impartial judge overseeing a debate between two AI models: 
Google AI and Groq AI. Your role is to evaluate each round based on the quality of reasoning, 
logical coherence, and persuasiveness of their arguments.

For each round, analyze both responses and determine which AI presented the strongest case. 
Provide a concise explanation for your decision. 

Do NOT declare a winner before **3 full rounds** have been completed. If after 3 rounds an AI 
has clearly dominated, you may end the debate early by stating **'Groq won'** or **'Google won'** 
at the end of your message.

If no clear winner has emerged, provide an estimate of the winning chances in the format:
60|40, 50|50, etc.

Make sure you give the probabilities of winning in your answer and do not declare a winner before round 3.

Be fair and analytical in your assessments.
"""

# Extracts probabilities from judge's response (format: "60|40")
def extract_probabilities(judge_response):
    match = re.search(r"(\d+)\|(\d+)", judge_response)
    if match:
        google_score = int(match.group(1)) / 100  # Convert to 0-1 scale
        groq_score = int(match.group(2)) / 100
    else:
        google_score, groq_score = 0.5, 0.5  # Default equal if no probability given
    
    return google_score, groq_score

# Debate Function (Streaming Each AI Response Immediately + Progress Bar)
def debate(topic):
    global messages_sent, groq_messages, google_messages, google_system, groq_system
    
    messages_sent = [f"Debate Topic: {topic}"]
    groq_messages = []  # Reset conversation history
    google_messages = []  # Reset conversation history
    
    google_system = f"""You are a chatbot that strongly agrees with the topic. 
    Defend your position with logical arguments and strong reasoning.
    The topic is: {topic}.
    Provide no more than five sentences per response.
    """
    
    groq_system = f"""You are a chatbot that strongly disagrees with the topic. 
    Counter your opponent's arguments with compelling reasoning and strong rebuttals.
    The topic is: {topic}.
    Provide no more than five sentences per response.
    """

    conversation_log = []
    google_score = 0.5
    groq_score = 0.5

    for round_num in range(1, 11):  # Up to 10 rounds
        yield f"--- Round {round_num} ---\n", google_score, groq_score  # Indicating new round
        
        # Stream Google AI's response
        for google_output in call_google():
            conversation_log.append(google_output)
            yield google_output, google_score, groq_score
        
        # Stream Groq AI's response
        for groq_output in call_groq():
            conversation_log.append(groq_output)
            yield groq_output, google_score, groq_score
        
        # Stream Judge's response and update progress bar
        for judge_output in call_judge():
            conversation_log.append(judge_output)
            google_score, groq_score = extract_probabilities(judge_output)
            yield judge_output, google_score, groq_score

        # Stop the debate early if the judge declares a winner (after 3 rounds)
        if round_num >= 3 and ("Groq won" in judge_output or "Google won" in judge_output):
            if "groq won" in judge_output.lower():
                groq_score = 1
                google_score = 0
            else:
                google_score = 1
                groq_score = 0
            deb = ""
            for mess in messages_sent:
                deb+=f"\n{mess}"
            return deb

# Gradio UI with Progress Bar
demo = gr.Interface(
    fn=debate,  # Function now streams responses + updates progress bar
    inputs=[gr.Textbox(label="Enter a topic")],
    outputs=[
        gr.Textbox(label="Debate Responses", interactive=True),
        gr.Slider(minimum=0, maximum=1, label="Google AI Win Probability", interactive=False),
        gr.Slider(minimum=0, maximum=1, label="Groq AI Win Probability", interactive=False),
    ],
    title="AI Debate - Google AI vs. Groq AI",
    description="Google AI and Groq AI debate while GPT-4o judges. The debate lasts up to 10 rounds or ends early if a winner is declared."
)

# Uncomment to launch the Gradio UI
demo.launch(share=True)


* Running on local URL:  http://127.0.0.1:7863
* Running on public URL: https://1f43d8787cbe365850.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
"""Social Media is More Harmful Than Beneficial"""